In [1]:
pip install optuna


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install meteostat


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install xgboost


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import optuna
import gc
import requests
import json
from sklearn.preprocessing import SplineTransformer
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_percentage_error,r2_score,mean_squared_error
from sklearn.model_selection import TimeSeriesSplit,cross_val_score,KFold
from datetime import date
from datetime import datetime
from sklearn.cluster import KMeans
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import os
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 50)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



/usr/local/python/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
!pip install importnb



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
# with the new api
from importnb import imports
with imports("ipynb"):
    import modules



In [8]:
class CFG:
    target = 'tüketim'
    train_path = 'data/train.csv'
    test_path = 'data/sample_submission.csv'
    med_path = 'data/med.csv'
    submission_path='data/sample_submission.csv'
    generation_path='data/TR_enerji_uretimi.csv'
    weatherfrom_wwo='data/izmir_weather_data.csv'
    meteostat_feature_columns = ['temp','dwpt','rhum','prcp','wdir','wspd','pres','coco']
    production_features = True
    consumption_features = False
    yuk_tahmin_plani_features = False

### Read Data

In [9]:
train=pd.read_csv(CFG.train_path).rename(columns={'Dağıtılan Enerji (MWh)':'tüketim'})
test=pd.read_csv(CFG.test_path).rename(columns={'Dağıtılan Enerji (MWh)':'tüketim'})
med=pd.read_csv(CFG.med_path)
submission=pd.read_csv(CFG.submission_path)
train = modules.timeseries_features(train.set_index('Tarih'))
test= modules.timeseries_features(test.set_index('Tarih'))
test['tüketim']=test['tüketim'].replace(0,np.nan)
train.index=pd.to_datetime(train.index)

### Train and Test data merged

In [10]:
all_data=pd.concat([train,test],axis=0)
all_data.index=pd.to_datetime(all_data.index)
med.Tarih=pd.to_datetime(med.Tarih)
# Gün içindeki elektrik kesinti süresinin kabul edilebilir limiti aştığı gün.
med['Tarih']=med['Tarih'].astype(str)

## External Data

In [11]:
weather=modules.get_meteostat_data(datetime(2018, 1, 1), datetime(2022, 9, 1),features=CFG.meteostat_feature_columns)[:-1]
generation=pd.read_csv(CFG.generation_path).iloc[:,1:]
generation.Tarih=pd.to_datetime(generation.Tarih)
generation=generation[['total','Tarih']]
# generation=real_time_generation()
consumption=modules.real_time_consumption()
con_pro=consumption.merge(generation,on='Tarih',how='left')
con_pro.set_index('Tarih',drop=True,inplace=True)
all_data=all_data.merge(weather,left_index=True,right_index=True)
all_data=all_data.merge(con_pro,left_index=True,right_index=True)

# ------------------------------------------------------------------------------------------------------------------------------------
yuk=pd.read_csv('data/YukTahmin.csv',encoding='unicode_escape').iloc[:40896,2]
all_data['yuk']=yuk.values
all_data.yuk=all_data.yuk.str.replace(',','').astype(float)
# ------------------------------------------------------------------------------------------------------------------------------------
ep=pd.read_csv('data/DENGESIZLIK.csv',encoding='unicode_escape').iloc[:40896,2:]
all_data['pozitif']=ep['Pozitif Dengesizlik Miktarý (MWh)'].values
all_data['pozitif']=all_data['pozitif'].str.replace(',','').astype(float)
# ------------------------------------------------------------------------------------------------------------------------------------
ue=pd.read_csv('data/UECM.csv',encoding='unicode_escape').iloc[:40896,2:]
ue['UEÇM (MWh)']=ue['UEÇM (MWh)'].str.replace(',','').astype(float)
all_data['ue']=ue['UEÇM (MWh)'].values

# ------------------------------------------------------------------------------------------------------------------------------------
weatherfrom_wwo=pd.read_csv(CFG.weatherfrom_wwo,index_col='date_time')[:'2022-08-31 23:00:00']
weatherfrom_wwo.index=pd.to_datetime(weatherfrom_wwo.index)
all_data=all_data.merge(weatherfrom_wwo[['pressure','humidity','uvIndex','WindGustKmph','windspeedKmph']],left_index=True,right_index=True)
all_index=all_data.index

### Seasonality features

In [12]:
all_data=modules.seasonality_features(all_data)
modules.info_data(all_data)

,null_rate,nunique,dtype
prcp,0.643559,65,float64
coco,0.078394,19,float64
tüketim,0.018192,40151,float64
wdir,0.004157,361,float64
pres,0.002152,400,float64
wspd,0.000073,123,float64
temp,0.000049,447,float64
dwpt,0.000049,331,float64
rhum,0.000049,91,float64
weekday,0.000000,7,int32


### Lag Features

In [13]:

lag_feature=[ 'temp', 'dwpt', 'rhum', 'prcp', 'wdir','wspd', 'pres', 'coco', 'consumption', 
             'total', 'yuk', 'pozitif', 'ue','pressure', 'humidity', 'uvIndex', 'WindGustKmph', 'windspeedKmph']

all_data,lagged_features=modules.lag_features(all_data,lag_feature,48,24) 
all_data=modules.diff_pct_features(all_data,lagged_features,[24,48])
all_data,rolled_features=modules.rolling_features(all_data,lag_feature,[6,12,18,24,30,36,42,48],['mean'])
all_data=modules.diff_pct_features(all_data,rolled_features,[24,48])
all_data=modules.rolling_shift_features(all_data,lag_feature,[24,7*24,14*24,30*24,3*30*24,6*30*24],['mean'],shift=24)

###  Changed some columns of dtypes to category

In [14]:
all_data=modules.is_categorical(all_data,all_data.columns,treshold_category=120/42000)
all_data.describe()
modules.info_data(all_data)

,tüketim,hour,dayofweek,month,year,dayofyear,dayofmonth,weekday,temp,dwpt,wdir,wspd,pres,consumption,total,yuk,pozitif,ue,lag_24_temp,lag_48_temp,lag_24_dwpt,lag_48_dwpt,lag_24_wdir,lag_48_wdir,lag_24_wspd,...,rolling_shift_24_mean_4320_pressure,rolling_shift_24_mean_24_humidity,rolling_shift_24_mean_168_humidity,rolling_shift_24_mean_336_humidity,rolling_shift_24_mean_720_humidity,rolling_shift_24_mean_2160_humidity,rolling_shift_24_mean_4320_humidity,rolling_shift_24_mean_24_uvIndex,rolling_shift_24_mean_168_uvIndex,rolling_shift_24_mean_336_uvIndex,rolling_shift_24_mean_720_uvIndex,rolling_shift_24_mean_2160_uvIndex,rolling_shift_24_mean_4320_uvIndex,rolling_shift_24_mean_24_WindGustKmph,rolling_shift_24_mean_168_WindGustKmph,rolling_shift_24_mean_336_WindGustKmph,rolling_shift_24_mean_720_WindGustKmph,rolling_shift_24_mean_2160_WindGustKmph,rolling_shift_24_mean_4320_WindGustKmph,rolling_shift_24_mean_24_windspeedKmph,rolling_shift_24_mean_168_windspeedKmph,rolling_shift_24_mean_336_windspeedKmph,rolling_shift_24_mean_720_windspeedKmph,rolling_shift_24_mean_2160_windspeedKmph,rolling_shift_24_mean_4320_windspeedKmph
count,40152.000000,40896.000000,40896.000000,40896.000000,40896.000000,40896.000000,40896.000000,40896.000000,40894.000000,40894.000000,40726.000000,40893.000000,40808.000000,40896.000000,40896.000000,40896.000000,40896.000000,40896.000000,40870.000000,40846.000000,40870.000000,40846.000000,40702.000000,40678.000000,40869.000000,...,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000,40872.000000
mean,1836.805287,11.500000,2.996479,6.238263,2019.856808,174.408451,15.725939,2.996479,18.246320,10.065000,196.245494,11.890209,1013.744124,34751.540653,34214.472276,34.552563,40858.318264,33.290014,18.241311,18.235340,10.058664,10.052784,196.210751,196.181695,11.893188,...,1014.251242,65.073268,65.078850,65.087515,65.133475,65.401517,65.654174,4.831523,4.825929,4.819855,4.804115,4.743430,4.689985,30.323269,30.327242,30.328988,30.288902,30.053010,29.892640,18.103125,18.105513,18.106859,18.087815,17.968311,17.878114
std,426.066085,6.922271,2.000315,3.381785,1.354686,103.395973,8.798882,2.000315,8.711412,5.845574,128.301229,8.045571,5.863431,5686.811429,5261.690238,5.561967,32096.707172,5.451963,8.711255,8.709873,5.841388,5.837900,128.316665,128.328874,8.046271,...,2.095491,9.579192,7.542693,6.869094,6.350607,5.647000,4.497441,1.511542,1.450121,1.428985,1.401021,1.273365,0.955817,12.961403,8.317637,6.824194,5.767108,3.923976,2.889754,8.506263,5.456675,4.468461,3.770452,2.774651,2.232585
min,870.183280,0.000000,0.000000,1.000000,2018.000000,1.000000,1.000000,0.000000,-4.000000,-15.000000,0.000000,0.000000,991.000000,15333.300000,15333.300000,17.237000,1.000040,14.578730,-4.000000,-4.000000,-15.000000,-15.000000,0.000000,0.000000,0.000000,...,1010.405941,36.750000,39.690476,42.229167,44.676389,48.764352,54.817824,1.000000,1.714286,2.142857,2.466667,2.833333,3.000000,6.416667,11.630952,14.282738,14.300000,14.300000,14.300000,3.458333,6.375000,6.823529,6.823529,6.823529,6.823529
25%,1499.165048,5.750000,1.000000,3.000000,2019.000000,86.000000,8.000000,1.000000,11.600000,6.400000,100.000000,5.400000,1009.500000,30488.020000,30273.022500,30.332000,1.935867,29.054720,11.600000,11.600000,6.400000,6.400000,100.000000,100.000000,5.400000,...,1012.445370,58.833333,60.428571,61.104167,61.538889,62.626736,63.620602,3.666667,3.571429,3.500000,3.433333,3.481481,3.802778,19.625000,24.005952,25.208333,25.711111,27.385185,28.497454,11.125000,13.934524,14.625000,14.988889,15.861458,16.841551
50%,1813.409221,11.500000,3.000000,6.000000,2020.000000,171.000000,16.000000,3.000000,18.000000,10.700000,186.000000,11.200000,1013.000000,34608.325000,34279.145000,34.500000,42658.500000,33.33117

,null_rate,nunique,dtype
lag_48_prcp,0.644733,65,category
rolling_shift_24_mean_168_prcp,0.644146,1449,float64
rolling_shift_24_mean_720_prcp,0.644146,2434,float64
rolling_shift_24_mean_2160_prcp,0.644146,4097,float64
rolling_shift_24_mean_4320_prcp,0.644146,6176,float64
lag_24_prcp,0.644146,65,category
rolling_shift_24_mean_24_prcp,0.644146,770,float64
rolling_shift_24_mean_336_prcp,0.644146,1879,float64
rolling_mean_36_prcp,0.643559,703,float64
rolling_mean_48_prcp,0.643559,931,float64


In [15]:
df_train=all_data[all_data.tüketim.notnull()]
df_test=all_data[all_data.tüketim.isnull()]
print(df_train.shape)
print(df_test.shape)

(40152, 327)
(744, 327)


### Target Encoding by some of seasonal columns

In [16]:
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim','hour','mean')
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim',['season','weekend','hour'],'mean')
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim',['season','weekday','prime_time','hour'],'mean')
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim',['season','prime_time'],'mean')

df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim','hour','sum')
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim',['season','weekend','hour'],'sum')
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim',['season','weekday','prime_time','hour'],'sum')

In [17]:
all_data=pd.concat([df_train,df_test])
all_data.drop(lag_feature,axis=1,inplace=True)
all_data.drop([
'date',
'dayofyear',
'dayofweek',
 # 'weekofyear'
# 'year'
],axis=1,inplace=True)

### Categorical features are labeled for xgboost

In [18]:
cat_features=all_data.select_dtypes('category').columns
le_fit={}
all_data[cat_features],le_fit=modules.label_data(feature=cat_features,data=all_data,le_fit=le_fit)
data_train=all_data[all_data.tüketim.notnull()]
test_sub=all_data[all_data.tüketim.isnull()]
test_sub.drop('tüketim',axis=1,inplace=True)
print(test_sub.shape)
print(data_train.shape)

(744, 312)
(40152, 313)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
test_sub

,hour,quarter,month,year,dayofmonth,weekday,wntr_month,month_label,season,weekend,working_hours,label_hour,prime_time,month_sin,month_cos,day_sin,day_cos,lag_24_temp,lag_48_temp,lag_24_dwpt,lag_48_dwpt,lag_24_rhum,lag_48_rhum,lag_24_prcp,lag_48_prcp,...,rolling_shift_24_mean_24_uvIndex,rolling_shift_24_mean_168_uvIndex,rolling_shift_24_mean_336_uvIndex,rolling_shift_24_mean_720_uvIndex,rolling_shift_24_mean_2160_uvIndex,rolling_shift_24_mean_4320_uvIndex,rolling_shift_24_mean_24_WindGustKmph,rolling_shift_24_mean_168_WindGustKmph,rolling_shift_24_mean_336_WindGustKmph,rolling_shift_24_mean_720_WindGustKmph,rolling_shift_24_mean_2160_WindGustKmph,rolling_shift_24_mean_4320_WindGustKmph,rolling_shift_24_mean_24_windspeedKmph,rolling_shift_24_mean_168_windspeedKmph,rolling_shift_24_mean_336_windspeedKmph,rolling_shift_24_mean_720_windspeedKmph,rolling_shift_24_mean_2160_windspeedKmph,rolling_shift_24_mean_4320_windspeedKmph,hour_te_x,season_weekend_hour_te_x,season_weekday_prime_time_hour_te_x,season_prime_time_te,hour_te_y,season_weekend_hour_te_y,season_weekday_prime_time_hour_te_y
0,0,2,8,2022,1,0,0,1,3,0,0,2,0,2,3,11,21,25.2,25.9,17.9,17.3,54,49,0,0,...,7.0,7.000000,6.785714,6.566667,5.856481,4.645370,37.500000,35.452381,43.568452,44.127778,35.286574,33.358333,20.083333,18.214286,23.241071,24.150000,19.643519,19.503241,1667.289193,1915.604283,1807.355303,2053.659530,2.789375e+06,107273.839858,110248.673508
1,1,2,8,2022,1,0,0,1,3,0,0,0,0,2,3,13,20,24.4,25.1,18.1,17.3,58,52,0,0,...,7.0,7.000000,6.785714,6.566667,5.857407,4.646296,37.083333,35.416667,43.532738,44.120833,35.283796,33.358565,19.958333,18.214286,23.229167,24.148611,19.640741,19.503009,1523.275694,1769.667279,1671.733959,2053.659530,2.548440e+06,99101.367651,101975.771483
2,2,2,8,2022,1,0,0,1,3,0,0,1,0,2,3,15,18,23.8,24.3,18.7,17.3,63,55,0,0,...,7.0,7.000000,6.785714,6.566667,5.858333,4.647222,36.750000,35.386905,43.500000,44.116667,35.282870,33.359259,19.708333,18.214286,23.217262,24.147222,19.637500,19.502546,1420.525514,1657.821245,1561.155129,2053.659530,2.376539e+06,92837.989728,95230.462891
3,3,2,8,2022,1,0,0,1,3,0,0,0,0,2,3,16,16,22.9,23.9,19.1,17.2,69,56,0,0,...,7.0,7.000000,6.785714,6.566667,5.859259,4.648148,36.416667,35.369048,43.476190,44.116667,35.283333,33.360648,19.416667,18.202381,23.202381,24.145833,19.635648,19.501620,1369.097812,1591.400905,1499.191051,2053.659530,2.290501e+06,89118.450696,91450.654112
4,4,2,8,2022,1,0,0,1,3,0,0,1,0,2,3,18,14,23.5,24.4,18.8,17.4,65,55,0,0,...,7.0,7.000000,6.785714,6.566667,5.860185,4.649074,36.041667,35.339286,43.452381,44.116667,35.283796,33.360880,19.166667,18.196429,23.190476,24.144444,19.635185,19.500463,1342.892269,1545.806167,1466.378634,2053.659530,2.246659e+06,86565.145378,89449.096686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,19,2,8,2022,31,2,0,1,3,1,0,0,0,2,3,1,11,26.3,27.1,22.6,20.2,70,56,0,0,...,7.0,6.119048,6.488095,6.633333,6.375926,5.279630,21.916667,17.946429,22.877976,32.298611,37.133333,33.526852,11.333333,9.636905,11.806548,16.991667,20.248148,19.068519,2119.241383,2292.534818,2344.213467,2053.659530,3.545491e+06,855115.487042,142997.021468
740,20,2,8,2022,31,2,0,1,3,1,0,1,1,2,3,3,13,25.6,26.3,22.7,20.8,74,62,0,0,...,7.0,6.125000,6.491071,6.633333,6.376389,5.280556,21.541667,17.875000,22.857143,32.255556,37.120833,33.527315,11.208333,9.613095,11.803571,16.973611,20.246296,19.069676,2101.625539,2284.982010,2335.804114,2283.712369,3.516020e+06,852298.289762,142484.050938
741,21,2,8,2022,31,2,0,1,3,1,0,0,1,2,3,5,15,25.0,26.1,22.5,20.4,76,61,0,0,...,7.0,6.130952,6.494048,6.633333,6.376852,5.281481,21.291667,17.863095,22.860119,32.220833,37.106944,33.528241,11.083333,9.601190,11.803571,16.956944,20.242593,19.070602,2040.558442,2279.977796,2333.846530,2283.712369,3.413854e+06,850431.717824,142364.638305
742,22,2,8,2022,31,2,0,1,3,1,0,

## Model Tuned

In [ ]:
target=data_train.tüketim
train_labeled=data_train.drop('tüketim',axis=1)
def objective(trial,data=train_labeled,target=target):
    
        param = {
            'base_score':0.5,
            'objective':'reg:squarederror',
            'booster':'gbtree', 
            'lambda': trial.suggest_float('lambda', 1e-3, 10),
            'alpha': trial.suggest_float('alpha', 1e-3, 10),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.7,0.8,0.9,1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.5,0.6,0.7,0.8,1.0]),
            'learning_rate': trial.suggest_categorical('learning_rate', [0.08,0.009,0.01,0.012,0.014,0.016,0.018,0.001,0.1,0.5]),
            'n_estimators':10000,
            'max_depth': trial.suggest_categorical('max_depth', [3,4,5,7,9,10]),
            'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 500),
            'tree_method':'hist',
            'eval_metric': "mape",
            'enable_categorical':True,
            'early_stopping_rounds':250
        } 

        best_iteration_list=[]
        score_list=[]
        model_list=[]
        fea_importance_list=[]
        x_train,x_val = data.iloc[:-2200], data.iloc[-2200:]  
        y_train,y_val = target.iloc[:-2200], target.iloc[-2200:]  
        model = xgb.XGBRegressor(**param)  
        model.fit(x_train,y_train,eval_set=[(x_train, y_train), (x_val, y_val)],verbose=0)
        best_iteration=model.best_iteration
        best_ntree_limit=model.best_ntree_limit
        val_pred=model.predict(x_val)
        model_list.append(model)
        score=mean_absolute_percentage_error(y_val, val_pred)
        score_list.append(score)
        fea_importance_list.append(model.feature_importances_)


        trial.set_user_attr('model',model_list)
        trial.set_user_attr('best_ntree_limit',best_ntree_limit)
        trial.set_user_attr('best_iteration',best_iteration)
        trial.set_user_attr("feature_importance", np.mean(fea_importance_list,axis=0))
        trial.set_user_attr("score_list", score_list)

        scores=np.mean(score)
        return scores



study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=85)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
gc.collect()

[I 2023-05-02 20:13:11,490] A new study created in memory with name: no-name-21ac13f2-5ff8-401f-85e8-0066ea39bbd0
[I 2023-05-02 20:14:02,650] Trial 0 finished with value: 0.041088735004382294 and parameters: {'lambda': 4.601768793731373, 'alpha': 1.2422721727110713, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.009, 'max_depth': 4, 'random_state': 2020, 'min_child_weight': 77}. Best is trial 0 with value: 0.041088735004382294.


#### BEST PARAMS 
**>params={
'lambda': 5.102807684567904, 
'alpha': 6.511737741485641,
'colsample_bytree': 0.9, 
'subsample': 0.7, 
'learning_rate': 0.012,
'max_depth': 10,
'random_state': 24,
'min_child_weight': 28,
'n_estimators': 1500,
'tree_method':'gpu_hist',
'enable_categorical':True}**


In [ ]:
param={
'lambda': 8.48551751148768, 'alpha': 2.366829085730496, 'colsample_bytree': 0.7, 'subsample': 0.8, 'learning_rate': 0.08, 'max_depth': 5, 'random_state': 48, 'min_child_weight': 24,
    'n_estimators': 1500,
    'tree_method':'gpu_hist',
        'enable_categorical':True
} 

all_models=[]
for i in [1200, 54,55]:
    param["random_state"] = i
    model = xgb.XGBRegressor(**param)  
    model.fit(train_labeled,target)
    all_models.append(model)

preds = [model.predict(test_sub)  for model in all_models]
mean_preds = np.mean(preds, axis=0)


best=pd.read_csv('/kaggle/input/bestresultgediz33/gediz_57.csv')
best['preds']=mean_preds
print(f'8.ay ortalama tüketim :{train[(train.year==2021) &(train.month==8)].tüketim.mean()}')
print(f'8.ay preds ortalama tüketim:{np.mean(mean_preds)}')
importances=model.feature_importances_
features=test_sub.columns
dum=pd.DataFrame(data={'features':features,'importances_rate':importances}).sort_values('importances_rate',ascending=False)
sns.set(rc={'figure.figsize':(30,15)})
g=sns.barplot(x=dum.features,y=dum.importances_rate)
g.set_xticklabels(g.get_xticklabels(), rotation=90);

In [ ]:
submission=pd.read_csv('/kaggle/input/gdz-elektrik-datathon-2023/sample_submission.csv')
submission['Dağıtılan Enerji (MWh)']=mean_preds
submission.set_index('Tarih').to_csv('/kaggle/working/gediz_57.csv')

In [ ]:
plt.plot(best['Dağıtılan Enerji (MWh)'])

In [ ]:
plt.plot(mean_preds)